<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T055731Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0ff4d189e4e149ce2436ef963107ad0bc3767447525f41c174309473e33c268b0106e0d281ba842cf02ce8714ed630c6a21f35e25b1327e2e4ebd8d2c23a1c196ab69c3141b8764efd2e46fe35139fadf9a1982c387ab03ec07de42f9eac35ce660d8eaccce784ffc62fd7367f39031d8c8f09064fd40ec43b063408cbab6ff3380c261e33b7c3ea81c9eb791168e065a9b428886382aab0f5dc2f9c4337241be130fc0035deb1d9687ecb4173375fcc6c2f420c988871dc46f7f321b1b81730e0004b2b9f93d4607131d2401112118c91f73dcb17acfa4b4cbdcc4e93c34f7ed2a19c4d0e99323e7363c413f61b4337c802a9b971c74e8df53b6dd4f78829d6,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T055731Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dda6d65c00671487785d64b4a14da2f185637b5c3e956c01cddcc438174d7ba4b7814092ccabc8a145814d2ed6bab61c42773800304c2a411cd8173de07c63fe959a4a04a653ed274d9e35ab4c4cc6706305fc9a3a5b65453a10f57281f79c08c4d2301e59db12d8174c538493264a4b12bfed43d79c9b5a2fdf200a18c6224e2811fdb63fb49ce8eba813385586455081636515b0ad1fe7ca1efb5c2eda6747fb8ed99906782f8bfce6e9651d71d82a80060aa634b229bdf0c238ce59310e85891c45a5b8db4f984e61d84f00adae474d5b298c2c0907578716ca3a51b4ed5d79f6ed2d7b74a952986d0f0ad60c9826a735327a521551a75070d3be016d2fdf9,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T055731Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D09d5b97a730cdb842a2355894bfb3e4830dc088f64481de5dda7f0db0311288cf2502bb9aa74f966636e0fa9a876873fba87d0b77cb81112104fe2bae4dccdf8185d2c9cfb618cd4992a6108a085a86182322f952f58feed0639b80ec4fab61ae88eadfe1074070640383a5eed572867322c127be3bb3c4056b84fa07c193d332ebffd56812705633a9d41c3011c9085b49720cb4fbf4d7f5b60b3fe0a8f25668ab012f1b9ea2ef1f003c3141559f1ac31b38aafcd64ecbf84e55d043c20feacfa4197bec7d99abc2c64ec8e3b35ca812708b6dd00760b33dd8dae579053cc40076ee0e9dcb699fa89e4fd00adb3d3340c0c17a541b6f8f530ca1571b99ae540'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) sponsored by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models y les code changes

## Requirements

In [ ]:
!pip install chromadb
# !pip install ydata-profiling
!pip install langchain
#!pip install pydantic
!pip install sentence-transformers
!pip install InstructorEmbedding

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [ ]:
##!pip install langchain-core==0.1.31
#!pip install chromadb==0.3.26
#!pip install ydata-profiling==4.6.1
#!pip install langchain==0.0.345
#!pip install pydantic==1.10.14
#!pip install sentence-transformers==2.6.1
#!pip install InstructorEmbedding==1.0.1

In [ ]:
#base
import os
import json
import re
import uuid
import time
import csv
import pandas as pd
from typing import *
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Classes

### Questions and answers data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        df_qa = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        df_qa = df_qa[df_qa['answer_score'] > 5].copy()
        df_qa = df_qa[df_qa['Tag']=='python'].copy()
        df_qa['Answer'] = df_qa['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        return df_qa

    def get_fine_tunning_data(self):
        """Returns a list of formatted user-assistant strings."""
        df_qa_data = self.qa_data()
        data = [
            f"<-change-of-interlocutor->user:\n{row['Question']}\n<-change-of-interlocutor->assistant:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data
    def get_qa_data(self):
        """Returns a list of records dictionaries """
        df_qa_data = self.qa_data()
        data = df_qa_data[
            [
                'Question',
                'Answer'
            ]
        ].to_dict(orient='records')
        return data


### Python Enhancement Proposals data

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text.replace('"""',"'") for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

In [ ]:
class Embeder:
    """
    Creates embeddings (numerical representations) of text documents for various tasks like semantic search and
    similarity comparison. Provides flexibility to choose between a large model for more comprehensive
    embeddings or a smaller, faster model.

    Attributes:

    large (bool): Flag indicating whether to use the large embedding model (default: True).
    model (str): Name of the embedding model to use.
    device (str): Device to use for computations (default: "cuda").
    """
    def __init__(
            self,
            large: bool = False
    ):
        """
        init(self, large: bool = True) -> None
            Initializes the class with the specified model size preference.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device: str = "cuda"
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(
                model_name=self.model
            )
        return embed

    def run(
            self,
            docs_list: list,
    ):
        """
        run(self, docs_list: list) -> List[List[float]]:
            Generates embeddings for the provided list of documents.
            Returns a list of lists, where each inner list represents the embedding vector for a document.
        """
        if docs_list is None:
            docs_list = ['']
        embed = self.instructor()

        return embed.embed_documents(docs_list)

### Prompt

In [ ]:
prompt_config = [{
    "prompt_id": "0.0.1",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 250,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.2",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}]

### Settings

In [ ]:
class Settings():
    def __init__(self):
        self.user_secrets = UserSecretsClient()
        self.CHUNK_SIZE = self.user_secrets.get_secret("CHUNK_SIZE")
        self.CHUNK_OVERLAP = self.user_secrets.get_secret("CHUNK_OVERLAP")
        self.CHROMA_NAME_INDEX = self.user_secrets.get_secret("CHROMA_NAME_INDEX")
        self.K = self.user_secrets.get_secret("K")
        self.NN_THRESHOLD = self.user_secrets.get_secret("NN_THRESHOLD")
        self.PROMPT_ID = self.user_secrets.get_secret("PROMPT_ID")
        self.prompt = [prompt for prompt in prompt_config if prompt['prompt_id']==self.PROMPT_ID][0]

### Retriever

In [ ]:
embeder = Embeder()
settings = Settings()
class Retriever:
    """ retriever
    Retrieves the embeddings from a vectorial database
    """
    def __init__(self):
        """ initialize the retriever
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX

    def load(self, data: list[dict]):
        """
        Loads documents from the initialized data and returns them as a list of Document objects.
        """
        documents = []
        for obj in data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def set(self, data: list[dict]):
        """
         update the vectorial database
        """
        data = self.load(data)
        splitter = self.text_splitter
        documents = splitter.split_documents(data)
        instructor = embeder.instructor()
        vector_db = Chroma.from_documents(
            documents=documents,
            embedding=instructor,
            persist_directory=self.index_name
        )
        vector_db.persist()

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
    ):
        """ retrieve the 2 more similar text chunks based on the message given
        """
        vectorial_db = Chroma(
            embedding_function=embeder.instructor(),
            persist_directory=self.index_name
        )
        res = vectorial_db.similarity_search_with_score(message, k=k)
        return [vector[0].page_content for vector in res if vector[1] < threshold]

### Message

In [ ]:
settings = Settings()
class Message:
    def __init__(
            self,
            role: str,
            content: str,
            timestamp: str = int(time.time()),
            prompt_id: str = settings.PROMPT_ID
    ):
        self.reply_id = str(uuid.uuid4())
        self.role = role
        self.content = content
        self.timestamp = timestamp
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))

    def reply(self):
        return {
            'reply_id': self.reply_id,
            'role': self.role,
            'content': self.content,
            'timestamp': self.timestamp
        }

    def system_reply(self):
        prompt = settings.prompt
        return {
            'reply_id': self.reply_id,
            'role': 'system',
            'content': prompt['system'],
            'timestamp': self.timestamp
        }
    def new_chat(self):
        init_chat = [self.system_reply, self.reply]
        json.dump(init_chat, open(self.file, 'w'))
        return init_chat
    def update(self):
        dict_history = json.load(open(self.file))
        dict_history.append(self.reply())
        json.dump(dict_history, open(self.file, 'w'))
    def restart_history(self):
        json.dump([], open(self.file, 'w'))

### Model Gemma

In [ ]:
class Gemma:
    def __init__(self):
        self.prompt = settings.prompt
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16
        ).cuda()
    def chat(self, context: str):
        # Use the model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
        outputs = self.gemma.generate(**input_ids, max_length=self.prompt['max_length'])
        return self.tokenizer.decode(outputs[0])

### Expert Python Agent/Assistant

In [ ]:
retriever = Retriever()
gemma = Gemma()

class Agent():
    def __init__(self):
        self.prompt = settings.prompt
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))
    def augmented_question(self, question: str):
        technical_documentation = retriever.query(question)
        if len(technical_documentation)>0:
            docs = self.prompt['technical_documentation']
            technical_documentation = '\n* '.join(technical_documentation)
            technical_documentation = technical_documentation[0]
            docs = docs.format(technical_documentation=technical_documentation)
            augmented_reply = f"""{question}{docs}"""
            return augmented_reply
        else:
            return question
    def memory(self, question: str):
        dict_history = json.load(open(self.file))
        message = Message(
                role='user',
                content=self.augmented_question(question)
            )
        if len(dict_history)>0:
            message.update()
            full_chat = json.load(open(self.file))
        else:
            full_chat = message.new_chat()
        return '\n'.join(
            [
                '\n<-change-of-interlocutor->'+reply['role']+': '+reply['content']
                for reply
                in full_chat
            ]
        )+'\n<-change-of-interlocutor->assistant:'

    def get_answer(self, full_response: str):
        answer_list = full_response.split('<-change-of-interlocutor->')
        pos_list = [
            pos
            for
            pos, answer
            in enumerate(answer_list)
            if question in answer
        ]
        tokened_answer = answer_list[pos_list[0]+1]
        answer = tokened_answer.split('assistant:')[1]
        message = Message(
            role='assistant',
            content=answer
        )
        message.update()
        return answer

    def chat(self, question: str, verbose: bool = False):
        memory = self.memory(question)
        full_response = gemma.chat(context=memory)
        if verbose:
            print(full_response)
        answer = self.get_answer(full_response)
        return answer



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Data

### fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_qa = PythonQAData.get_qa_data()
print(python_qa[1486])

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_fine_tunning_data()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][:1000]
)

## few-shot data

In [ ]:
few_shot_list=[7,6009,12,36,8034,130,141,537,1057,5042]
for i, q in enumerate(few_shot_list):
    print('_'*100,f'\n{i}.',python_qa[q])

## RAG data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

corpus lenght: 644 
section <n> lenght: 7009 
sample text:

 e ‘fractions’ module.


Rationale
While sometimes slower and more memory intensive (in general,
unboundedly so) rational arithmetic captures more closely the
mathematical ideal of numbers, and tends to have behavior which is
less surprising to newbies.  Though many Python implementations of
rational numbers have been written, none of these exist in the
core, or are documented in any way.  This has made them much less
accessible to people who are less Python-savvy.


RationalType
There will be a new numeric type added called RationalType.  Its
unary operators will do the obvious thing.  Binary operators will
coerce integers and long integers to rationals, and rationals to
floats and com
CPU times: user 48.2 s, sys: 615 ms, total: 48.8 s
Wall time: 55.2 s


In [ ]:
%%time
peps = Peps()
data = peps.jsonl_format()
data[-1]['text'][:2000]

CPU times: user 50.7 s, sys: 593 ms, total: 51.3 s
Wall time: 57.2 s


'\n\nPython Enhancement Proposals\n\nPython » \nPEP Index » \nPEP 8102\n\n\n\n\n\nToggle light / dark / auto colour theme\n\n\n\n\nPEP 8102 – 2021 Term Steering Council election\n\nAuthor:\nEwa Jodlowska <ewa at python.org>, Ee Durbin <ee at python.org>, Joe Carey <joe at python.org>\nSponsor:\nBrett Cannon <brett at python.org>\nStatus:\nFinal\nType:\nInformational\nTopic:\nGovernance\nCreated:\n29-Oct-2020\n\n\n\nTable of Contents\nAbstract\nElection Administration\nSchedule\nCandidates\nVoter Roll\nElection Implementation\nConfiguration\nQuestions\nQuestion 1\n\n\n\n\nResults\nCopyright\nComplete Voter Roll\nActive Python core developers\n\n\n\n\n\nAbstract\nThis document describes the schedule and other details of the December\n2020 election for the Python steering council, as specified in\nPEP 13. This is the steering council election for the 2021 term.\n\n\nElection Administration\nThe steering council appointed the\nPython Software Foundation\nDirector of Infrastructure, Ee Durb

# RAG

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["hello I'm goku"])[0]
print(len(vec))
vec[:10]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384
CPU times: user 2.67 s, sys: 1.07 s, total: 3.74 s
Wall time: 1min


[-0.1933707594871521,
 0.08631032705307007,
 0.2352217584848404,
 0.4100143313407898,
 -0.14344947040081024,
 0.19855117797851562,
 0.46806424856185913,
 -0.08672317862510681,
 -0.2187383770942688,
 0.1548473834991455]

In [ ]:
%%time
retriever = Retriever()
retriever.set(data)

CPU times: user 4min 19s, sys: 8.43 s, total: 4min 28s
Wall time: 3min 36s


In [ ]:
retriever = Retriever()
question = """
Does the community advantage of Git outweigh the familiarity of Mercurial for CPython?
"""
retriever.query(question)[1]

'Version Control System\nCurrently the CPython and supporting repositories use Mercurial.  As a\nmodern distributed version control system, it has served us well since\nthe migration from Subversion.  However, when evaluating the VCS we\nmust consider the capabilities of the VCS itself as well as the\nnetwork effect and mindshare of the community around that VCS.\nThere are really only two real options for this, Mercurial and Git.\nThe technical capabilities of the two systems are largely equivalent,\ntherefore this PEP instead focuses on their social aspects.\nIt is not possible to get exact numbers for the number of projects or\npeople which are using a particular VCS, however we can infer this by\nlooking at several sources of information for what VCS projects are\nusing.\nThe Open Hub (previously Ohloh) statistics [1] show that\n37% of the repositories indexed by The Open Hub are using Git (second\nonly to Subversion which has 48%) while Mercurial has just 2%, beating'

# Chatbot

## Loading few-shot to the memory

In [ ]:
file = 'data/history.json'
message = Message(
    role='sistem',
    content='foo'
)
init_chat = [message.system_reply()]
json.dump(init_chat, open(file, 'w'))
for i, q in enumerate(few_shot_list):
    sample = python_qa[q]
    mesage = Message(
        role='user',
        content=sample['Question']
    )
    reply = mesage.update()
    mesage = Message(
        role='assitant',
        content=sample['Answer']
    )
    reply = mesage.update()

history = json.load(open(file))
history

[{'reply_id': '88da8bd5-ea40-47b5-9864-2cc9f9343291',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n        ',
  'timestamp': 1712284189},
 {'reply_id': '1b3881ee-b26f-4efe-ab4e-e5e7c591f772',
  'role': 'user',
  'content': "How do I use Python's itertools.groupby()?",
  '

### Testing the RAG system

In [ ]:
agent = Agent()
print(agent.memory("""
Version Control System
Currently the CPython and supporting repositories use Mercurial. As a modern distributed version control system, it has served us well since the migration from Subversion. However, when evaluating the VCS we must consider the capabilities of the VCS itself as well as the network effect and mindshare of the community around that VCS.

There are really only two real options for this, Mercurial and Git. The technical capabilities of the two systems are largely equivalent, therefore this PEP instead focuses on their social aspects.

It is not possible to get exact numbers for the number of projects or people which are using a particular VCS, however we can infer this by looking at several sources of information for what VCS projects are using.

The Open Hub (previously Ohloh) statistics [1] show that 37% of the repositories indexed by The Open Hub are using Git (second only to Subversion which has 48%) while Mercurial has just 2%,
"""))


<-change-of-interlocutor->system: 
Act as a Python programming language expert assistant 
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary 
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        

<-change-of-interlocutor->user: How do I use Python's itertools.groupby()?

<-change-of-interlocutor->assitant: As Sebastjan said, you first have to sort your data. This is important.

The part I didn't get is that in the exa

# GPC - 0.0.1

In [ ]:
question

'\nVersion Control System\nCurrently the CPython and supporting repositories use Mercurial. As a modern distributed version control system, it has served us well since the migration from Subversion. However, when evaluating the VCS we must consider the capabilities of the VCS itself as well as the network effect and mindshare of the community around that VCS.\n\nThere are really only two real options for this, Mercurial and Git. The technical capabilities of the two systems are largely equivalent, therefore this PEP instead focuses on their social aspects.\n\nIt is not possible to get exact numbers for the number of projects or people which are using a particular VCS, however we can infer this by looking at several sources of information for what VCS projects are using.\n\nThe Open Hub (previously Ohloh) statistics [1] show that 37% of the repositories indexed by The Open Hub are using Git (second only to Subversion which has 48%) while Mercurial has just 2%, \nDoes the community advan

In [ ]:
%%time
agent = Agent()
question = 'what is the latest python version that is currently lunched'
print('*'*500,'\n final answer:',agent.chat(question, verbose=True))

<bos>
<-change-of-interlocutor->system: 
Act as a Python programming language expert assistant 
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary 
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        

<-change-of-interlocutor->user: How do I use Python's itertools.groupby()?

<-change-of-interlocutor->assitant: As Sebastjan said, you first have to sort your data. This is important.

The part I didn't get is that in th

In [ ]:
%%time
agent = Agent()
print(agent.chat('what is python??', verbose=True))

<bos>
<-change-of-interlocutor->system: 
Act as a Python programming language expert assistant 
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary 
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        

<-change-of-interlocutor->user: How do I use Python's itertools.groupby()?

<-change-of-interlocutor->assitant: As Sebastjan said, you first have to sort your data. This is important.

The part I didn't get is that in th

IndexError: list index out of range

In [ ]:
answer_list="""
<-change-of-interlocutor->assitant: You can use date from the <a href="http://docs.python.org/library/datetime.html">datetime module to find the first Sunday in a year and then keep adding seven days, generating new Sundays:

from datetime import date, timedelta

def allsundays(year):
   d = date(year, 1, 1)                    # January 1st
   d += timedelta(days = 6 - d.weekday())  # First Sunday
   while d.year == year:
      yield d
      d += timedelta(days = 7)

for d in allsundays(2010):
   print d



<-change-of-interlocutor->user: what is python??
<-change-of-interlocutor->assistant: Python is a programming language. It is a high-level, interpreted, object-oriented programming language. It is designed to have as few implementation dependencies as possible. It is a multi-paradigm language, which means it supports imperative, object-oriented, and functional programming paradigms.

""".split('<-change-of-interlocutor->')
pos_list = [
      pos
     for
       pos, answer

            in enumerate(answer_list)
     if question in answer
     ]
pos_list

[]

In [ ]:
question

'what is the latest python version that is currently lunched'

In [ ]:
%%time
agent = Agent()
print(agent.chat('my name is juan nice to meet you!!!'))

  The latest version is 3.5.2.

You can check it by typing python --version in the terminal.





CPU times: user 1min 13s, sys: 75 ms, total: 1min 13s
Wall time: 1min 15s


In [ ]:
history = json.load(open(file))
history

[{'reply_id': '88da8bd5-ea40-47b5-9864-2cc9f9343291',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n        ',
  'timestamp': 1712284189},
 {'reply_id': '1b3881ee-b26f-4efe-ab4e-e5e7c591f772',
  'role': 'user',
  'content': "How do I use Python's itertools.groupby()?",
  '

# Fine-Tunning

## GPC - 0.1.0

In [ ]:
#for i in range(100000000000000000):
#    1*1

KeyboardInterrupt: 